In [1]:
import pandas as pd
import numpy as np


In [9]:
# Path to raw data
file_path = "../data/NIFTY 50_minute.csv"

# Load CSV
df = pd.read_csv(file_path)

# Preview data
df.head()


,date,open,high,low,close,volume
0,2015-01-09 09:15:00,8285.45,8295.90,8285.45,8292.10,0
1,2015-01-09 09:16:00,8292.60,8293.60,8287.20,8288.15,0
2,2015-01-09 09:17:00,8287.40,8293.90,8287.40,8293.90,0
3,2015-01-09 09:18:00,8294.25,8300.65,8293.90,8300.65,0
4,2015-01-09 09:19:00,8300.60,8301.30,8298.75,8301.20,0


In [11]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 975321 entries, 0 to 975320
Data columns (total 6 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   date    975321 non-null  object 
 1   open    975321 non-null  float64
 2   high    975321 non-null  float64
 3   low     975321 non-null  float64
 4   close   975321 non-null  float64
 5   volume  975321 non-null  int64  
dtypes: float64(4), int64(1), object(1)
memory usage: 44.6+ MB


In [13]:
df.columns = [col.lower().strip() for col in df.columns]
df.columns


Index(['date', 'open', 'high', 'low', 'close', 'volume'], dtype='object')

In [17]:
# Convert datetime column
df["datetime"] = pd.to_datetime(df["date"])

# Sort chronologically
df = df.sort_values("datetime")

# Set datetime as index
df.set_index("datetime", inplace=True)

df.head()


,date,open,high,low,close,volume
datetime,,,,,,
2015-01-09 09:15:00,2015-01-09 09:15:00,8285.45,8295.90,8285.45,8292.10,0
2015-01-09 09:16:00,2015-01-09 09:16:00,8292.60,8293.60,8287.20,8288.15,0
2015-01-09 09:17:00,2015-01-09 09:17:00,8287.40,8293.90,8287.40,8293.90,0
2015-01-09 09:18:00,2015-01-09 09:18:00,8294.25,8300.65,8293.90,8300.65,0
2015-01-09 09:19:00,2015-01-09 09:19:00,8300.60,8301.30,8298.75,8301.20,0


In [19]:
df = df[~df.index.duplicated(keep="first")]


In [21]:
df_5m = df.resample("5T").agg({
    "open": "first",
    "high": "max",
    "low": "min",
    "close": "last",
    "volume": "sum"
})


C:\Users\Hp\AppData\Local\Temp\ipykernel_10552\4273940384.py:1: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  df_5m = df.resample("5T").agg({


In [23]:
# Drop incomplete candles
df_5m.dropna(inplace=True)

df_5m.head()


,open,high,low,close,volume
datetime,,,,,
2015-01-09 09:15:00,8285.45,8301.30,8285.45,8301.20,0
2015-01-09 09:20:00,8300.50,8303.00,8293.25,8301.00,0
2015-01-09 09:25:00,8301.65,8302.55,8286.80,8294.15,0
2015-01-09 09:30:00,8294.10,8295.75,8280.65,8288.50,0
2015-01-09 09:35:00,8289.10,8290.45,8278.00,8283.45,0


In [25]:
# Check time delta consistency
df_5m.index.to_series().diff().value_counts().head()


datetime
0 days 00:05:00    192461
0 days 17:50:00      1978
2 days 17:50:00       473
1 days 17:50:00        72
3 days 17:50:00        58
Name: count, dtype: int64

In [29]:
output_path = "../data/nifty_spot_5min.csv"
df_5m.to_csv(output_path)

print(f"Saved 5-minute data to {output_path}")


Saved 5-minute data to ../data/nifty_spot_5min.csv
